In [ ]:
import os


api_key = os.getenv('MY_API_KEY')
api_base = os.getenv('MY_API_BASE')

#langchain
* prompt,prompt模板
* 加载外部文件，怎么解决文件token数超出chatgpt limit限制
* 怎么管理/使用外部工具
* 集成向量数据库

https://python.langchain.com/docs/get_started/introduction

langchain就是设计了一套框架/API,来解决大模型应用开发当中碰到的问题，做到开箱即用

直接进入代码

In [ ]:
!pip3 install langchain==0.0.335

In [ ]:
from langchain.prompts.chat import ChatPromptTemplate
from langchain.chat_models.openai import ChatOpenAI

template = "You are a helpful assistant that translate {input_lang} to {output_lang}"
human_template = "{text}"

chat_template = ChatPromptTemplate.from_messages([
    ("system",template),
    ("human",human_template)
])
chain = chat_template | ChatOpenAI(openai_api_base=api_base,openai_api_key=api_key)
mag = chain.invoke({"input_lang":"English","output_lang":"Chinese","text":"I love programming"})
print(mag)

这段代码，主要涉及到两个组件，一个是llm模型，ChatOpenAI,一个是prompt template；“|”是langchain的表达式(LCEL ,langchain expresion language),表达式是langchian新版的特性，可以更加方便/灵活的组件执行链

下面一个一个模块开始介绍

# Model I/O
一个经典的执行链基本上包括三个最小组成部分
* prompt/输入管理
* 模型预测推理
* 输出的parser

## prompt管理，PromptTemplate
主要有两类
* PromptTemplate
* ChatPromptTemplate

PromptTemplate
主要用于非chat场景，无状态

In [ ]:
from langchain.prompts import PromptTemplate
prompt_template = PromptTemplate.from_template(
    "Tell me a {adjective} joke about {content}"
)
print(prompt_template.format(adjective="funny",content="chikens"))

ChatPromptTemplate
主要用于chat类场景，有状态

In [ ]:
from langchain.prompts import ChatPromptTemplate
chat_prompt_template = ChatPromptTemplate.from_messages(
    [
        ("system","You are a helpful assistant AI bot, Your name is {name}"),
        ("human","Hello, how are you doing?"),
        ("ai","I'm doing greate!"),
        ("human","{user_input}"),
    ]
)
print(chat_prompt_template.format_messages(name="Bob",user_input="What's your name?"))

In [ ]:
from langchain.prompts import ChatPromptTemplate,HumanMessagePromptTemplate,SystemMessagePromptTemplate
from langchain.schema.messages import SystemMessage,AIMessage,HumanMessage
chat_prompt_template = ChatPromptTemplate.from_messages(
    [
        SystemMessagePromptTemplate.from_template("You are a helpful assistant AI bot, Your name is {name}"),
        HumanMessage(content="Hello, how are you doing?"),
        AIMessage(content="I'm doing greate!"),
        HumanMessagePromptTemplate.from_template("{user_input}")
    ]
)
print(chat_prompt_template.format_messages(name="Bob",user_input="What's your name?"))

当然有些prompt是随时可变，有些prompt是初始化一次就好的

In [ ]:
prompt_template = PromptTemplate.from_template("translate words to {lang}:{words}")
partial_template = prompt_template.partial(lang = "Chinese")

print(partial_template.format(words="I lobe programming"))

## FewShotChatMessagePromptTemplate

In [ ]:
from langchain.prompts import FewShotChatMessagePromptTemplate

exmaples = [
    {"input":"2+2","output":"4"},
    {"input":"2+6","output":"8"},
    {"input":"10+10","output":"20"},
]

example_template = ChatPromptTemplate.from_messages(
    [
        ("human","{input}"),
        ("ai","{output}")
    ]
)

few_shot_template = FewShotChatMessagePromptTemplate(
    example_prompt=example_template,
    examples=exmaples
)

final_template = ChatPromptTemplate.from_messages(
    [
        ("system","You're a wondrous wizard of math"),
        few_shot_template,
        ("human","{input}")
    ]
)
print(final_template.format(input="99+99"))

展开来说，few-shop的目的是为了让大模型从上下文中学习，更好的完成任务；为了使用更好的例子，可以用example-selector,结合向量数据库一起使用

In [ ]:
!pip3 install tiktoken

In [ ]:
from langchain.embeddings import OpenAIEmbeddings
from langchain.prompts import SemanticSimilarityExampleSelector
from langchain.vectorstores.chroma import Chroma

exmaples = [
    {"input":"2+2","output":"4"},
    {"input":"2+6","output":"8"},
    {"input":"10+10","output":"20"},
    {"input":"tell me a joke","output":"haha"}
]

# 将例子存入向量数据库
to_vec= [" ".join(example.values()) for example in exmaples]
embeddings = OpenAIEmbeddings(openai_api_base=api_base,openai_api_key=api_key)
vector_store = Chroma.from_texts(to_vec,embeddings,metadatas=exmaples)

# 检索最优的例子
exmaple_selector = SemanticSimilarityExampleSelector(
    vectorstore=vector_store,
    k = 1,
)

exmaple_selector.select_examples({"input":"1+1"})

few_shot_template = FewShotChatMessagePromptTemplate(
    input_variables=["input"],
    example_selector=exmaple_selector,
    example_prompt=ChatPromptTemplate.from_messages(
        [
            ("human","{input}"),
            ("ai","{output}")
        ]
    )
)

final_prompt = ChatPromptTemplate.from_messages([
     ("system","You're a wondrous wizard of math"),
     few_shot_template,
     ("human","{input}")
    ]
)
print(final_prompt.format(input="99+99"))

当然，exmaple_selector是可以自定义的，准确来说所有组件都可以自定义；

# 大模型/LLM
* LLM
* ChatModel
llm直接用于推理预测，chat接口，多role交互

ChatModel

In [ ]:
chat = ChatOpenAI(api_key=api_key,api_base=api_base)

messages = [
    SystemMessage(content="You're a helpful assistant"),
    HumanMessage(content="tell me a joke")
]
chat.invoke(messages)

LLM

In [ ]:
from langchain.llms.openai import OpenAI
llm = OpenAI(api_base=api_base,api_key=api_key)
llm.invoke("tell me a joke")

因为资源是很贵的，所以缓存也是当然的

In [ ]:
from langchain.cache import InMemoryCache
from langchain.globals import set_llm_cache
from langchain.callbacks import get_openai_callback

# 设置缓存
set_llm_cache(InMemoryCache())
llm = OpenAI(api_base=api_base,api_key=api_key)

with get_openai_callback() as cb:
    print(llm.invoke("tell me a joke"))
    print(cb)

with get_openai_callback() as cb:
    print(llm.invoke("tell me a joke"))
    print(cb)




当然，缓存也可以从内存缓存数据库

In [ ]:
from langchain.cache import SQLiteCache
set_llm_cache(SQLiteCache(database_path=".langchain.db"))

也可以使用huggingface上的开源模型

In [ ]:
# from langchain.llms.huggingface_hub import HuggingFaceHub

# llm = HuggingFaceHub(repo_id="",huggingfacehub_api_token="")
# llm.invoke()

# output parser
就是模型输出的解析。开发大模型应用必备组件，通常在大模型进行格式化输出后，接着就可以执行业务逻辑，如使用外部外部工具等

In [ ]:
from langchain.output_parsers.json import SimpleJsonOutputParser

json_prompt = PromptTemplate.from_template(
    "Return a JSON object with an 'answer' that answer the following question:{question}"
)
json_parser = SimpleJsonOutputParser()
json_chain = json_prompt | OpenAI(openai_api_base=api_base,openai_api_key=api_key) | json_parser
json_chain.invoke({"question":"what's hugging face"})

 自定义output parser

In [ ]:
from langchain.schema import BaseOutputParser

class CustomOutPutParser(BaseOutputParser):
    """parser llm output"""

    def parse(self, text: str) -> str:
        """parse the output call"""
        return text.strip().split(" ")
    
json_prompt = PromptTemplate.from_template(
    "Return a JSON object with an 'answer' that answer the following question:{question}"
)
custom_parser = CustomOutPutParser()
json_chain = json_prompt | OpenAI(openai_api_base=api_base,openai_api_key=api_key) | custom_parser
json_chain.invoke({"question":"what's hugging face"})

这一期讲了langchain执行链最基本的三个组件
* promptTemplate
* llm/chat
* ouputparser

# Retrieval
RAG(Retrival argument generate)检索增强生成。
* 文本导入
    * token过长怎么办？
    * 对文本怎么切分处理？
* 外部工具
* 向量数据库
    * Chroma

## 文档加载

In [ ]:
!pip3 install pypdf

## langchain提供了多种文档加载器
* csv
* file directory
* html
* json
* markdown
* pdf

In [ ]:
from langchain.document_loaders import CSVLoader,DirectoryLoader,UnstructuredHTMLLoader,JSONLoader,UnstructuredMarkdownLoader,PyPDFLoader,TextLoader

# csv
#loader = CSVLoader(file_path="")
#data = loader.load()

# file directory
#loader = DirectoryLoader('',glob=".txt",loader_cls=TextLoader)

# html
#loader = UnstructuredHTMLLoader(file_path="")

# json
#loader = JSONLoader(file_path="")

# markdown
#loader = UnstructuredMarkdownLoader(file_path="")

# pdf_loader
loader = PyPDFLoader(file_path="./transformer.pdf")
data = loader.load()

print(data)



## Document transformers文本转换/分割
大模型token限制，需要分割文档

In [ ]:
!pip3 install tiktoken==0.3.3

tiktoken是openai开发的分词器

In [ ]:
from langchain.text_splitter import CharacterTextSplitter

loader = PyPDFLoader(file_path="./transformer.pdf")

text_slitter = CharacterTextSplitter.from_tiktoken_encoder(
    chunk_size = 100,chunk_overlap=10,separator = " "
)
pages = loader.load_and_split(text_slitter)
print(len(pages))

怎么对一个大文本进行摘要总结呢？如果不适用langchain已有的组件，最简单的方式就是便利每一个文本块总结。

In [ ]:
from langchain.prompts.prompt import PromptTemplate
from langchain.llms.openai import OpenAI
from langchain.text_splitter import CharacterTextSplitter
import time

prompt_template = PromptTemplate.from_template(
    "Here's your first summary:{pre_response}"
    "Now add to it based on the following context:{context}"
)

llm = OpenAI(api_key=api_key,api_base=api_base)

def dfs(docs,cur_doc_index,summary):
    if cur_doc_index == len(docs):
        print(f"final summary: {summary}")
        return
    
    document = docs[cur_doc_index]
    prompt_str = prompt_template.format_prompt(pre_response = summary,context=document.page_content)
    summary = llm.invoke(prompt_str)
    time.sleep(3)
    print(f"each summary {cur_doc_index}: {summary}")
    dfs(docs,cur_doc_index + 1,summary)

loader = PyPDFLoader(file_path="./transformer.pdf")
text_slitter = CharacterTextSplitter.from_tiktoken_encoder(
    chunk_size = 200,chunk_overlap=10,separator = " "
)
pages = loader.load_and_split(text_slitter)
dfs(pages[2:5],0,"")


langchain提供了文本总结的组件，分别有
* Stuff
* Refine
* Map Reduce
* Map Re-Rank

## Stuff
stuff就是暴力总结，将文本一股脑传给大模型，不管有没有超出limit限制


In [ ]:
## Stuff
from langchain.prompts import PromptTemplate
from langchain.schema.prompt_template import format_document
from langchain.schema import Document

text = """Nuclear power in space is the use of nuclear power in outer space, typically either small fission systems or radioactive decay for electricity or heat. Another use is for scientific observation, as in a Mössbauer spectrometer. The most common type is a radioisotope thermoelectric generator, which has been used on many space probes and on crewed lunar missions. Small fission reactors for Earth observation satellites, such as the TOPAZ nuclear reactor, have also been flown.[1] A radioisotope heater unit is powered by radioactive decay and can keep components from becoming too cold to function, potentially over a span of decades.[2]

The United States tested the SNAP-10A nuclear reactor in space for 43 days in 1965,[3] with the next test of a nuclear reactor power system intended for space use occurring on 13 September 2012 with the Demonstration Using Flattop Fission (DUFF) test of the Kilopower reactor.[4]

After a ground-based test of the experimental 1965 Romashka reactor, which used uranium and direct thermoelectric conversion to electricity,[5] the USSR sent about 40 nuclear-electric satellites into space, mostly powered by the BES-5 reactor. The more powerful TOPAZ-II reactor produced 10 kilowatts of electricity.[3]

Examples of concepts that use nuclear power for space propulsion systems include the nuclear electric rocket (nuclear powered ion thruster(s)), the radioisotope rocket, and radioisotope electric propulsion (REP).[6] One of the more explored concepts is the nuclear thermal rocket, which was ground tested in the NERVA program. Nuclear pulse propulsion was the subject of Project Orion.[7]

Regulation and hazard prevention[edit]
After the ban of nuclear weapons in space by the Outer Space Treaty in 1967, nuclear power has been discussed at least since 1972 as a sensitive issue by states.[8] Particularly its potential hazards to Earth's environment and thus also humans has prompted states to adopt in the U.N. General Assembly the Principles Relevant to the Use of Nuclear Power Sources in Outer Space (1992), particularly introducing safety principles for launches and to manage their traffic.[8]

Benefits

Both the Viking 1 and Viking 2 landers used RTGs for power on the surface of Mars. (Viking launch vehicle pictured)
While solar power is much more commonly used, nuclear power can offer advantages in some areas. Solar cells, although efficient, can only supply energy to spacecraft in orbits where the solar flux is sufficiently high, such as low Earth orbit and interplanetary destinations close enough to the Sun. Unlike solar cells, nuclear power systems function independently of sunlight, which is necessary for deep space exploration. Nuclear-based systems can have less mass than solar cells of equivalent power, allowing more compact spacecraft that are easier to orient and direct in space. In the case of crewed spaceflight, nuclear power concepts that can power both life support and propulsion systems may reduce both cost and flight time.[9]

Selected applications and/or technologies for space include:

Radioisotope thermoelectric generator
Radioisotope heater unit
Radioisotope piezoelectric generator
Radioisotope rocket
Nuclear thermal rocket
Nuclear pulse propulsion
Nuclear electric rocket
"""
doc_prompt = PromptTemplate.from_template("{page_content}")
docs = [Document(page_content=split) for split in text.split("\n")]
chain = {"content" : lambda docs: "\n".join(format_document(doc,doc_prompt) for doc in docs)} | PromptTemplate.from_template("Summarize the following content:\n{content}")
print(chain.invoke(docs))

## Refine
这个其实就和例子中的递归本质上一样的，先总结第一个文本，然后带这上一个总结以及下一个文本合并总结，以此递归

In [ ]:
from functools import partial
from operator import itemgetter
from langchain.schema import StrOutputParser

text = """Nuclear power in space is the use of nuclear power in outer space, typically either small fission systems or radioactive decay for electricity or heat. Another use is for scientific observation, as in a Mössbauer spectrometer. The most common type is a radioisotope thermoelectric generator, which has been used on many space probes and on crewed lunar missions. Small fission reactors for Earth observation satellites, such as the TOPAZ nuclear reactor, have also been flown.[1] A radioisotope heater unit is powered by radioactive decay and can keep components from becoming too cold to function, potentially over a span of decades.[2]

The United States tested the SNAP-10A nuclear reactor in space for 43 days in 1965,[3] with the next test of a nuclear reactor power system intended for space use occurring on 13 September 2012 with the Demonstration Using Flattop Fission (DUFF) test of the Kilopower reactor.[4]

After a ground-based test of the experimental 1965 Romashka reactor, which used uranium and direct thermoelectric conversion to electricity,[5] the USSR sent about 40 nuclear-electric satellites into space, mostly powered by the BES-5 reactor. The more powerful TOPAZ-II reactor produced 10 kilowatts of electricity.[3]

Examples of concepts that use nuclear power for space propulsion systems include the nuclear electric rocket (nuclear powered ion thruster(s)), the radioisotope rocket, and radioisotope electric propulsion (REP).[6] One of the more explored concepts is the nuclear thermal rocket, which was ground tested in the NERVA program. Nuclear pulse propulsion was the subject of Project Orion.[7]

Regulation and hazard prevention[edit]
After the ban of nuclear weapons in space by the Outer Space Treaty in 1967, nuclear power has been discussed at least since 1972 as a sensitive issue by states.[8] Particularly its potential hazards to Earth's environment and thus also humans has prompted states to adopt in the U.N. General Assembly the Principles Relevant to the Use of Nuclear Power Sources in Outer Space (1992), particularly introducing safety principles for launches and to manage their traffic.[8]

Benefits

Both the Viking 1 and Viking 2 landers used RTGs for power on the surface of Mars. (Viking launch vehicle pictured)
While solar power is much more commonly used, nuclear power can offer advantages in some areas. Solar cells, although efficient, can only supply energy to spacecraft in orbits where the solar flux is sufficiently high, such as low Earth orbit and interplanetary destinations close enough to the Sun. Unlike solar cells, nuclear power systems function independently of sunlight, which is necessary for deep space exploration. Nuclear-based systems can have less mass than solar cells of equivalent power, allowing more compact spacecraft that are easier to orient and direct in space. In the case of crewed spaceflight, nuclear power concepts that can power both life support and propulsion systems may reduce both cost and flight time.[9]

Selected applications and/or technologies for space include:

Radioisotope thermoelectric generator
Radioisotope heater unit
Radioisotope piezoelectric generator
Radioisotope rocket
Nuclear thermal rocket
Nuclear pulse propulsion
Nuclear electric rocket
"""

# 对第一个文本总结
first_prompt = PromptTemplate.from_template("Summarize this content:\n{context}")

# 读取doc
doc_prompt = PromptTemplate.from_template("{page_content}")
partial_format_doc = partial(format_document,prompt = doc_prompt)

#执行链
summary_chain = {"context":partial_format_doc} | first_prompt | llm

# 循环所有页
refine_template = PromptTemplate.from_template(
    "Here's your first summary:{pre_response}"
    "Now add to it based on the following context:{context}"
)

# 执行链
refine_chain = {"pre_response":itemgetter("pre_response"),"context":lambda x:partial_format_doc(x['doc'])} | refine_template | llm

def loop(docs):
    summary = summary_chain.invoke(docs[0])
    # 轮询剩下的
    for i , doc in enumerate(docs[1:]):
        summary = refine_chain.invoke({"pre_response":summary,"doc":doc})
        print(summary)
    return summary

docs = [Document(page_content=split) for split in text.split("\n")]
print(loop(docs[:3]))



## Map Reduce
分为两个阶段，首先是map阶段，对每一个文档进行总结，reduce阶段将之前的所有总结合并
## Map Re-Rank
这种主要用在问答场景
map阶段对每一个文本进行打分，将分数最高的文本作为提示词进行作答

有更加方便，开箱即用的API

In [ ]:
from langchain.chains.summarize import load_summarize_chain

loader = PyPDFLoader("./transformer.pdf")
text_slitter = CharacterTextSplitter.from_tiktoken_encoder(chunk_size=100,chunk_overlap=0,separator=" ")
docs = loader.load_and_split(text_slitter)

chain = load_summarize_chain(llm,chain_type="refine",verbose=True)

chain.run(docs[:3])

# embedding & vector store & retriver

In [ ]:
from langchain.embeddings import OpenAIEmbeddings

embeddings_model = OpenAIEmbeddings(api_key=api_key,api_base=api_base)

embeddings = embeddings_model.embed_documents(
    [
        "Hi,Dog name is haha",
        "Cat name's hehe",
        "My name is xixi"
    ]
)
embedding_query = embeddings_model.embed_query("what's your name?")
embedding_query[:2]

embeddings_model同样也支持cache

In [ ]:
from langchain.storage import InMemoryStore
from langchain.embeddings import CacheBackedEmbeddings
store = InMemoryStore()

embeddings_model = OpenAIEmbeddings(api_key=api_key,api_base=api_base)

embedder = CacheBackedEmbeddings.from_bytes_store(embeddings_model,store)

In [ ]:
from langchain.vectorstores.chroma import Chroma

pdf_loader = PyPDFLoader("./transformer.pdf")
text_slitter = CharacterTextSplitter.from_tiktoken_encoder(chunk_size=100,chunk_overlap=0,separator=" ")
pdf_docs = pdf_loader.load_and_split(text_slitter)
db = Chroma.from_documents(documents=pdf_docs[:5],embedding=OpenAIEmbeddings(api_key=api_key,api_base=api_base))
query = "what's transformer?"
result= db.similarity_search(query)
print(result)

然后是检索API

In [ ]:
from langchain.vectorstores.chroma import Chroma

pdf_loader = PyPDFLoader("./transformer.pdf")
text_slitter = CharacterTextSplitter.from_tiktoken_encoder(chunk_size=100,chunk_overlap=0,separator=" ")
pdf_docs = pdf_loader.load_and_split(text_slitter)
db = Chroma.from_documents(documents=pdf_docs[:5],embedding=OpenAIEmbeddings(api_key=api_key,api_base=api_base))

retriever = db.as_retriever()

print(retriever.invoke("what's transformer?"))

最后，结合rag+llm做一个demo

In [ ]:
text = "Hi,Dog name is haha\nCat name's hehe\nMy name is xixi\n"
text_slitter = CharacterTextSplitter.from_tiktoken_encoder(chunk_size=100,chunk_overlap=0,separator=" ")
texts = text_slitter.split_text(text)

em = OpenAIEmbeddings(api_key=api_key,api_base=api_base)
db = Chroma.from_texts(texts,em)
retriever = db.as_retriever()

retriever_docs = retriever.invoke("what's your name?")

template = "Answer the question based only on the following context:\n{context}\nQuestion:{question}"

prompt = ChatPromptTemplate.from_template(template)
model = ChatOpenAI(api_key=api_key,api_base=api_base)

chain = {"context":retriever_docs} | prompt | model | StrOutputParser()

chain.invoke("name?")

当然langchain也提供了多种的检索器，最后介绍一种比较有用的，WebResearchRetriever

In [ ]:
from langchain.retrievers.web_research import WebResearchRetriever
from langchain.utilities.bing_search import BingSearchAPIWrapper
from langchain.chains import RetrievalQAWithSourcesChain

vector_store= Chroma(embedding_function=OpenAIEmbeddings(api_key=api_key,base_url=api_base),persist_directory="./chrome_db")

llm = ChatOpenAI(api_key=api_key,base_url=api_base)

search = BingSearchAPIWrapper(bing_search_url="https://api.bing.microsoft.com/v7.0/search",bing_subscription_key=os.getenv('BING_API_KEY'))

wrr = WebResearchRetriever.from_llm(vectorstore=vector_store,llm=llm,search=search)

user_input = "How do LLM Agent work?"
qa_chain = RetrievalQAWithSourcesChain.from_chain_type(llm,retriever=wrr)
result = qa_chain.invoke({"question":user_input})

# next
* chain的原理，以及tools user
* agent模式，原理